In [ ]:
import json
import numpy as np

from flask import Flask, request

app = Flask(__name__)


@app.route('/rank', methods =['POST'])
def ranking():
    j = request.json
    inverted_index = j['data']
    documents = j['documents']
    words = j['words']
    
    # create tf_idf -> {docID: tf_idf}
    tf_idf = {key: [] for key in documents}
    
    for docID in documents:        
        tf_idf_for_doc = 0
        
        for term in words:   
            idf_for_term = idf[term]
            list_of_dicts = inverted_index[term]
            print(list_of_dicts)
            print(docID)
            tf_for_term = [np.log(1 + (len(doc['position_text']) + len(doc['position_title'])) / doc['doc_len']) 
                           for doc in list_of_dicts if doc['id'] == docID]
         
            tf_idf_for_doc += tf_for_term[0] * idf_for_term
            
        tf_idf[docID] = tf_idf_for_doc
   

    # sorting documents
    ranked = sorted(tf_idf.items(), key=lambda kv: kv[1], reverse=True)
    ranked = [d[0] for d in ranked]
      
    return json.dumps({'status':'ok', 'ranked': ranked})

@app.route("/rank/idf", methods=['POST'])
def refresh_idf():
    global idf
    j = request.json
    idf = j['data']
    return json.dumps({'status':'ok'})

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=13541)